In [0]:
!pip install -U -q PyDrive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, utils, svm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.utils import shuffle
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
# Load Libs for Google Collab
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [0]:
# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
listed = drive.ListFile({'q': "title contains '.csv' and 'XXX' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))
  
X_downloaded = drive.CreateFile({'id': 'XXX',
                                'parents': 'XXX'})
X_downloaded.GetContentFile('X.csv')
y_downloaded = drive.CreateFile({'id': 'XXX',
                                'parents': 'XXX'})
y_downloaded.GetContentFile('y.csv')


title final_gps.csv, id 1xvb7fXI3tljcmQHpLL-zFN_sy5tZudT0
title y.csv, id 1XllzfPQjn0fRIAYaeaHNthW4Bse34vHQ
title X.csv, id 1gu40JLV4--waNeEhFuMrSjo4pcLZQHLG
title dataset_resampled.csv, id 11JrgqGJ3fZBjPxnOln3XP-_Zyy9_Oltd
title gps_resampled.csv, id 1ZlRhPOcKzvozDuegHqeZADttUCyy0yk7
title Magnetometer_resampled.csv, id 1N99JLcBTlyMA3EvBwPOJRvwFEbRFjujA
title Gyroscope_resampled.csv, id 11TMVVqvzqQqWPL75bA6ivBdmSiYh60Kn
title Accelerometer_resampled.csv, id 12i3VsbCgJbXfwZ6JM5pPXzDnPMJZhg9R
title y_test.csv, id 1RHr926Y4B8AYscEosY5oM9-odRjym5I8
title y_train.csv, id 1JYgOsx258fyTy0IZRX5-5IphR4anUglE
title final_gyr.csv, id 12t2rFXZAdVdj-KhhQcD6HETm2j4o_o9G
title final_mag.csv, id 1scKZyWvepwgtoBo1jj9SsvAWLZk9VqZk
title X_test.csv, id 1V_UGsG1wQMjfE-gBIxibreN23dnV7M6b
title X_train.csv, id 1Nvn2kvxeVALss3kFzph7CwPI4lnqRoIf
title final_acc.csv, id 1QzQmjP7KzB6DGn0zlbjoRt8oc1-dkEWY
title gps_filtered.csv, id 1Vi6PK8pyqT4pa8hBJ3sCApNq0zdLowl_


In [0]:
X = pd.read_csv('X.csv', sep='\t', decimal=',', encoding='utf-8', index_col=0)
y = pd.read_csv('y.csv', sep='\t', decimal=',', encoding='utf-8', index_col=0)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=52)

In [0]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

34932 34932
8733 8733


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [0]:
classifier = RandomForestClassifier()

In [0]:
parameters = {'n_estimators': [10, 100, 1000], 'max_depth': [3, 6, 9], 'max_features': ['auto']}

In [0]:
print(len(X_train.as_matrix()))
print(len(y_train.as_matrix().ravel().T))
model = GridSearchCV(classifier, parameters, n_jobs=-1, scoring='accuracy', cv=10, verbose=4)
model.fit(X_train.as_matrix(), y_train.as_matrix().ravel().T)

34932
34932
Fitting 10 folds for each of 9 candidates, totalling 90 fits
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV]  max_depth=3, max_features=auto, n_estimators=10, score=0.700886 -   0.8s
[CV]  max_depth=3, max_features=auto, n_estimators=10, score=0.701288 -   0.7s
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV]  max_depth=3, max_features=auto, n_estimators=10, score=0.723240 -   0.7s
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV]  max_depth=3, max_features=auto, n_estimators=10, score=0.720092 -   0.7s
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV]  max_depth=3, max_features=auto, n_estimators=10, score=0.710564 -   0.8s
[CV] max_depth=3, max_features=auto, n_estimators=10 .................
[CV]  max_depth=3, max_features=aut

[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.4min


[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.728183 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.752433 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.729250 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.734898 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.728314 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.732608 - 1.6min
[CV] max_depth=3, max_features=auto, n_estimators=1000 ...............
[CV]  max_depth=3, max_features=auto, n_estimators=1000, score=0.728314 - 1.6min
[CV] ma

[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 46.0min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [10, 100, 1000], 'max_depth': [3, 6, 9], 'max_features': ['auto']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=4)

In [0]:
model.grid_scores_

[mean: 0.71845, std: 0.01259, params: {'max_depth': 3, 'max_features': 'auto', 'n_estimators': 10},
 mean: 0.73620, std: 0.00817, params: {'max_depth': 3, 'max_features': 'auto', 'n_estimators': 100},
 mean: 0.73511, std: 0.00797, params: {'max_depth': 3, 'max_features': 'auto', 'n_estimators': 1000},
 mean: 0.85815, std: 0.00908, params: {'max_depth': 6, 'max_features': 'auto', 'n_estimators': 10},
 mean: 0.88042, std: 0.00529, params: {'max_depth': 6, 'max_features': 'auto', 'n_estimators': 100},
 mean: 0.88644, std: 0.00458, params: {'max_depth': 6, 'max_features': 'auto', 'n_estimators': 1000},
 mean: 0.94435, std: 0.00528, params: {'max_depth': 9, 'max_features': 'auto', 'n_estimators': 10},
 mean: 0.95520, std: 0.00382, params: {'max_depth': 9, 'max_features': 'auto', 'n_estimators': 100},
 mean: 0.95420, std: 0.00309, params: {'max_depth': 9, 'max_features': 'auto', 'n_estimators': 1000}]

In [0]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [0]:
print("Best paramaters: ", model.best_params_)
print("Accuracy score: ", accuracy*100)

Best paramaters:  {'max_depth': 9, 'max_features': 'auto', 'n_estimators': 100}
Accuracy score:  95.24791022558114


In [0]:
LABELS = ['Bike', 'Car', 'Jogging', 'Metro', 'Tram', 'Train', 'Walking']

cfm = pd.DataFrame(confusion_matrix(y_test, y_pred))

cfm.index = LABELS
cfm.columns = LABELS

cfm

,Bike,Car,Jogging,Metro,Tram,Train,Walking
Bike,1126,2,0,8,1,22,9
Car,1,1142,0,3,9,0,1
Jogging,0,2,1019,0,0,0,3
Metro,6,7,0,1243,108,49,5
Tram,8,11,0,54,1061,10,3
Train,8,13,1,28,20,1366,3
Walking,7,3,1,8,0,1,1361
